In [79]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.datasets import make_classification

In [80]:
if torch.cuda.is_available():
    print("GPU available.")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
elif torch.mps.is_available():
    print("MPS available.")
    print(f"Using MPS: {torch.mps.device_count()}")
    device = torch.device("mps")
else:
    print("Using CPU.")
    device = torch.device("cpu")

device = torch.device("cpu")

torch.set_default_device(device=device)

MPS available.
Using MPS: 1


In [81]:
# Creating dataset

x, y = make_classification(
    n_samples=12,        # Number of samples
    n_features=2,        # Number of features
    n_informative=2,     # Number of informative features
    n_redundant=0,       # Number of redundant features
    n_classes=2,         # Number of classes
    random_state=42,     # For reproductibility
)

In [82]:
x = torch.tensor(x, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device)

In [83]:
class CustomDataset(Dataset):

    def __init__(self, feartures, labels):
        self.features = feartures
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [84]:
dataset = CustomDataset(x, y)

In [85]:
dataloader = DataLoader(dataset=dataset, batch_size=3, shuffle=True)

In [86]:
for batch_features, batch_labels in dataloader:

    print(batch_features)
    print(batch_labels)
    print("-" * 50)

tensor([[-0.5527, -2.5911],
        [ 2.2757, -0.1670],
        [ 3.0016, -0.1474]])
tensor([0., 1., 1.])
--------------------------------------------------
tensor([[ 1.5116, -0.2226],
        [-0.0863,  0.9132],
        [-1.0306,  0.2967]])
tensor([0., 1., 1.])
--------------------------------------------------
tensor([[-0.9379, -1.3595],
        [-1.0936, -0.6333],
        [ 0.1023,  0.8010]])
tensor([0., 0., 1.])
--------------------------------------------------
tensor([[ 1.4669,  0.4553],
        [-1.6591,  2.8911],
        [ 0.2005, -0.9657]])
tensor([0., 1., 0.])
--------------------------------------------------


In [87]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [88]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [89]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [90]:
x_train_tensor = torch.from_numpy(x_train).to(dtype=torch.float32, device=device)
x_test_tensor = torch.from_numpy(x_test).to(dtype=torch.float32, device=device)
y_train_tensor = torch.from_numpy(y_train).to(dtype=torch.float32, device=device)
y_test_tensor = torch.from_numpy(y_test).to(dtype=torch.float32, device=device)

In [91]:
class CancerDataset(Dataset):

    def __init__(self, feartures, labels):
        self.features = feartures
        self.labels = labels

    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [92]:
train_dataset = CancerDataset(x_train_tensor, y_train_tensor)
test_dataset = CancerDataset(x_test_tensor, y_test_tensor)

In [93]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [94]:
class NeuralNetwork(nn.Module):

    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=3, device=device),
            nn.ReLU(),
            nn.Linear(in_features=3, out_features=1, device=device),
            nn.Sigmoid(),
        )
    
    def forward(self, features):
        out = self.network(features)
        return out

In [95]:
lr = 0.1
epochs = 1000

In [105]:
model = NeuralNetwork(x_train_tensor.shape[1]).to(device=device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr)

loss_function = nn.BCELoss()

In [106]:
for epoch in range(epochs):
    for batch_features, batch_labels in train_loader:
        y_pred = model(batch_features)
        loss = loss_function(y_pred, batch_labels.view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")

Epoch: 1, Loss: 0.6197615265846252
Epoch: 2, Loss: 0.5268048644065857
Epoch: 3, Loss: 0.24264369904994965
Epoch: 4, Loss: 0.20525319874286652
Epoch: 5, Loss: 0.16420944035053253
Epoch: 6, Loss: 0.08108653128147125
Epoch: 7, Loss: 0.053193412721157074
Epoch: 8, Loss: 0.10267382860183716
Epoch: 9, Loss: 0.05230791121721268
Epoch: 10, Loss: 0.3086850345134735
Epoch: 11, Loss: 0.08979503810405731
Epoch: 12, Loss: 0.022370746359229088
Epoch: 13, Loss: 0.016022812575101852
Epoch: 14, Loss: 0.09543914347887039
Epoch: 15, Loss: 0.025396283715963364
Epoch: 16, Loss: 0.02010170742869377
Epoch: 17, Loss: 0.011669384315609932
Epoch: 18, Loss: 0.0018276566406711936
Epoch: 19, Loss: 0.003944388125091791
Epoch: 20, Loss: 0.0009329589665867388
Epoch: 21, Loss: 0.010786147788167
Epoch: 22, Loss: 0.0018729480216279626
Epoch: 23, Loss: 0.19289268553256989
Epoch: 24, Loss: 0.025507645681500435
Epoch: 25, Loss: 0.026140540838241577
Epoch: 26, Loss: 0.00594195444136858
Epoch: 27, Loss: 0.007892879657447338


In [104]:
model.eval()
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.9).float()

        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy: .4f}')

Accuracy:  0.6094
